In [1]:
import os
import re
import math
import copy
import warnings
import datetime
import numpy as np
import pandas as pd

In [2]:
import sys
sys.path.append("C:/Users/begas/Project/python/SmartFarm/5. SRC") # 다른 경로의 Class 불러올 때 사용
import common_preprocess_tools as prep
import model_tools             as mdl

C:\Users\begas\Anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### 기본 셋팅하기

In [3]:
# -----------------------------------------------|
# ----- Step 1. Settings ------------------------
# -----------------------------------------------|
os.chdir("C:/Users/begas/Project/python/SmartFarm/")
os.getcwd()

home_path  = os.getcwd()
data_path  = os.path.join(home_path,"1. DAT")
save_path  = os.path.join(home_path,"2. OUT")
model_path = os.path.join(home_path,"3. MODEL")

data_files = os.path.join(data_path,os.listdir(data_path)[6])
print(data_files)

working_data = '20220916'

C:\Users\begas\Project\python\SmartFarm\1. DAT\22_이엔티DB_신천농장_외부.csv


### 데이터 불러오기

In [4]:
'''
data_files  : Raw 데이터 위치
time_grp    : 시간대별 통계량 요약 기준 단위(ex : if time_grp = 60, then 60분 단위로 통계량 요약)
timr_window : 시차 (예측시간으로부터 window_v시간 전의 데이터를 설명변수로 사용)
'''
_prep = prep.common_preprocess(data_files = data_files, time_grp = 60, time_window = [12,24])

In [5]:
_prep.load_fn()

>> Raw 데이터 호출 및 단일값 컬럼 제거
모든 값이 단일값인 변수 제거 : 온도/환기팬


,시간,암모니아,황화수소,습도
0,2022-01-01 00:23:00,0.0,0.1,49.0
1,2022-01-01 00:23:00,0.0,0.1,49.0
2,2022-01-01 00:23:00,0.0,0.2,49.0
3,2022-01-01 00:23:00,0.0,0.1,49.0
4,2022-01-01 00:23:00,0.0,0.1,49.0
...,...,...,...,...
42365,2022-01-16 15:32:00,2.4,0.1,43.0
42366,2022-01-16 15:32:00,2.4,0.1,43.0
42367,2022-01-16 15:32:00,2.4,0.1,43.0
42368,2022-01-16 15:32:00,2.4,0.1,43.0


### 학습 데이터 전처리 하기

In [6]:
_prep.preprocess_fn()

>> 이상치, 결측치 처리 및 시간대별 요약 통계량 생성
Step1. 이상치를 허용범위 내로 보정
Step2. 시간 변수를 60 분 단위로 변경
Step3. 시간별 요약통계량 데이터 생성
Step4. 특정 시간대의 데이터가 비어있을 시 해당 시간대 생성
- 22개의 시간대가 비어있습니다. 해당 시간대를 생성합니다.
Step5. 선형 보간법 적용


,시간,암모니아_mean,황화수소_mean,습도_mean,암모니아_min,황화수소_min,습도_min,암모니아_max,황화수소_max,습도_max,암모니아_std,황화수소_std,습도_std
0,2022-01-01 00:00:00,0.083333,0.140000,49.000000,0.0,0.1,49.0,0.4,0.2,49.0,0.163128,0.049195,0.000000
1,2022-01-01 01:00:00,0.013333,0.137500,49.016667,0.0,0.1,49.0,0.4,0.2,50.0,0.072103,0.048615,0.128556
2,2022-01-01 02:00:00,0.000000,0.135000,49.183333,0.0,0.1,49.0,0.0,0.2,50.0,0.000000,0.047897,0.388562
3,2022-01-01 03:00:00,0.000000,0.140000,50.000000,0.0,0.1,50.0,0.0,0.2,50.0,0.000000,0.049195,0.000000
4,2022-01-01 04:00:00,0.000000,0.135833,49.866667,0.0,0.1,49.0,0.0,0.2,50.0,0.000000,0.048152,0.341360
...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,2022-01-16 11:00:00,1.223333,0.113333,50.375000,1.2,0.1,50.0,1.6,0.2,51.0,0.094142,0.034136,0.486153
372,2022-01-16 12:00:00,1.286667,0.116667,50.000000,1.2,0.1,50.0,1.6,0.2,50.0,0.165480,0.037424,0.000000
373,2022-01-16 13:00:00,1.456667,0.110000,47.375000,1.2,0.1,47.0,1.6,0.2,50.0,0.192609,0.030126,0.996317
374,2022-01-16 14:00:00,1.913333,0.106667,46.416667,1.6,0.1,46.0,2.0,0.2,47.0,0.165480,0.025049,0.495074


### 학습 - 분석용 데이터 생성

In [7]:
_prep.train_anal_dat_fn()

>> 학습 - 분석용 데이터 생성[종속, 설명(시차변수) 생성]
Step1. 목표변수: 암모니아, 황화수소 단위별 max 값
Step2. 시차 변수 생성
Step3. 학습/검증 데이터 7:3으로 분할
Step4. 단일 값만 가지는 설명변수 제거


{'train_dat':            predict_time  y_암모니아  y_황화수소           base_time  x_암모니아_mean_12  \
 0   2022-01-02 00:00:00     1.2     0.2 2022-01-01 00:00:00        1.056667   
 1   2022-01-02 01:00:00     1.6     0.2 2022-01-01 01:00:00        1.200000   
 2   2022-01-02 02:00:00     1.6     0.2 2022-01-01 02:00:00        1.700000   
 3   2022-01-02 03:00:00     1.6     0.2 2022-01-01 03:00:00        2.490000   
 4   2022-01-02 04:00:00     1.6     0.2 2022-01-01 04:00:00        2.703333   
 ..                  ...     ...     ...                 ...             ...   
 242 2022-01-12 02:00:00     0.0     0.2 2022-01-11 02:00:00        1.206667   
 243 2022-01-12 03:00:00     0.0     0.2 2022-01-11 03:00:00        1.786667   
 244 2022-01-12 04:00:00     0.0     0.2 2022-01-11 04:00:00        1.976667   
 245 2022-01-12 05:00:00     0.0     0.2 2022-01-11 05:00:00        1.546667   
 246 2022-01-12 06:00:00     0.0     0.2 2022-01-11 06:00:00        1.193333   
 
      x_황화수소_mean_12  x_습

### 모델링

In [8]:
_model = mdl.model('y_암모니아',_prep.train_anal_dat)

In [9]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    _model.modeling_mlr()
    _model.modeling_rf()
    _model.modeling_xgb()
    _model.modeling_lgb()
    _model.modeling_ann()
    _model.modeling_lstm()

Model Type is MLR
Model Type is Random Forest
Model Type is XGBoost
Model Type is LightGBM
Model Type is ANN
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with 